In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fonm
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

pd.describe_option('display.max_seq_items', None)

'display.max_seq_items : int or None\n    when pretty-printing a long sequence, no more then `max_seq_items`\n    will be printed. If items are omitted, they will be denoted by the\n    addition of "..." to the resulting string.\n\n    If set to None, the number of items to be printed is unlimited.\n    [default: 100] [currently: 100]'

In [2]:
ki=pd.read_excel('d:\\data2\\한국투자증권\\한국투자증권.xlsx')

In [3]:
ki

,고객구분코드,주문일자,고객성별구분코드,동일나이군구분코드,주소(시도),주문시간대,상품번호,상품명,거래소구분코드,매도매수구분코드,...,총체결금액,전일종가,당일시가,일중고가,일중저가,당일종가,거래수량,거래대금,상한가여부,하한가여부
0,A0001,20200128,1,30,경기,1,00000A091970,나노캠텍,3,2,...,75900,3215,3710,4125,3660,4095,17266597,67323312855,N,N
1,A0001,20200130,1,30,경기,3,00000A045060,오공,3,2,...,154200,7690,8150,8380,7360,7920,15686931,125404000000,N,N
2,A0001,20200313,1,30,경기,1,00000A253840,수젠텍,3,2,...,118000,13600,12200,12250,10150,10750,3731106,40782765600,N,N
3,A00010,20200220,1,20,경남,3,00000A009190,대양금속보통주,2,2,...,49070,3615,3615,3675,3475,3505,635410,2250893355,N,N
4,A000100,20200211,2,40,서울,1,00000A001740,SK네트웍스보통주,2,1,...,14625,4780,4800,4885,4800,4850,1263089,6112026870,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845935,A0009999,20200331,1,20,충북,2,00000A043200,파루,3,2,...,303000,1995,2035,2135,1990,2030,2764564,5682482755,N,N
845936,A0009999,20200325,1,20,충북,2,00000A068760,셀트리온제약,3,1,...,507200,57300,61700,70600,58700,60300,8892944,560929000000,N,N
845937,A0009999,20200330,1,20,충북,1,00000A038290,마크로젠,3,2,...,970000,38900,40050,40800,37500,40400,1912985,75631708300,N,N
845938,A0009999,20200323,1,20,충북,1,00000A000660,에스케이하이닉스보통주,2,1,...,553600,74800,69700,71600,68000,69400,8037426,560505000000,N,N


In [4]:
ki.columns

Index(['고객구분코드', '주문일자', '고객성별구분코드', '동일나이군구분코드', '주소(시도)', '주문시간대', '상품번호',
       '상품명', '거래소구분코드', '매도매수구분코드', '주문구분코드', '실주문단가', '주문수량', '총체결수량',
       '총체결금액', '전일종가', '당일시가', '일중고가', '일중저가', '당일종가', '거래수량', '거래대금',
       '상한가여부', '하한가여부'],
      dtype='object')

In [5]:
ki.groupby('고객구분코드')['주문수량'].count()

고객구분코드
A0001        758
A00010        15
A000100       14
A0001000      11
A00010000     27
            ... 
A0009995      15
A0009996      20
A0009997       6
A0009998      59
A0009999      37
Name: 주문수량, Length: 24989, dtype: int64

In [6]:
ki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845940 entries, 0 to 845939
Data columns (total 24 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   고객구분코드     845940 non-null  object
 1   주문일자       845940 non-null  int64 
 2   고객성별구분코드   845940 non-null  int64 
 3   동일나이군구분코드  845940 non-null  int64 
 4   주소(시도)     845940 non-null  object
 5   주문시간대      845940 non-null  int64 
 6   상품번호       845940 non-null  object
 7   상품명        845940 non-null  object
 8   거래소구분코드    845940 non-null  int64 
 9   매도매수구분코드   845940 non-null  int64 
 10  주문구분코드     845940 non-null  int64 
 11  실주문단가      845940 non-null  int64 
 12  주문수량       845940 non-null  int64 
 13  총체결수량      845940 non-null  int64 
 14  총체결금액      845940 non-null  int64 
 15  전일종가       845940 non-null  int64 
 16  당일시가       845940 non-null  int64 
 17  일중고가       845940 non-null  int64 
 18  일중저가       845940 non-null  int64 
 19  당일종가       845940 non-null  int64 
 20  거래수량

In [31]:
st = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
st #한국거래소에서 상장법인목록을 엑셀로 다운로드

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
1,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,KEC,92220,반도체 제조업,반도체 제조,2006-10-16,12월,박남규 김학남,http://www.kec.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...
2448,타임기술,318660,소프트웨어 개발 및 공급업,"ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역",2021-06-24,12월,주양효,http://timett.co.kr,경상남도
2449,툴젠,199800,자연과학 및 공학 연구개발업,"유전자가위 (유전체 교정 도구) 및 이를 이용한 유전자세포치료제, 고부가가치 가축,...",2014-06-25,12월,"김영호, 이병화(각자 대표이사)",http://www.toolgen.com,서울특별시
2450,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2451,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [32]:
st1=st[['종목코드','업종','상장일','지역']]
st1 #필요 컬럼만 추출

st1.rename(columns={'종목코드':'상품번호'},inplace=True) # 한투 데이터와 merge를 위해 기준 컬럼명 일치

st1.상품번호=st1.상품번호.astype(str)
st1.info()
st2=st1.copy() #사본

st2.상품번호=st1.상품번호.apply(lambda x: '0'*5+'A'+'0'*(6-len(x))+x) #한투의 상품번호 컬럼과 동일한형태

st3=st2.copy() #사본

st3.상품번호=st3.상품번호.apply(lambda x:x[:-1]) # 우선주와 보통주 합치기위해 마지막 숫자 (보통주 0,우선주5 또는 문자)


st3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2453 entries, 0 to 2452
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품번호    2453 non-null   object
 1   업종      2453 non-null   object
 2   상장일     2453 non-null   object
 3   지역      2453 non-null   object
dtypes: object(4)
memory usage: 76.8+ KB


C:\Users\wanw9\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\wanw9\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,상품번호,업종,상장일,지역
0,00000A00484,고무제품 제조업,1976-05-21,부산광역시
1,00000A15566,1차 비철금속 제조업,2013-05-15,부산광역시
2,00000A00125,상품 종합 도매업,1976-06-26,서울특별시
3,00000A29487,건물 건설업,2018-06-12,서울특별시
4,00000A09222,반도체 제조업,2006-10-16,서울특별시
...,...,...,...,...
2448,00000A31866,소프트웨어 개발 및 공급업,2021-06-24,경상남도
2449,00000A19980,자연과학 및 공학 연구개발업,2014-06-25,서울특별시
2450,00000A22267,자연과학 및 공학 연구개발업,2015-07-28,서울특별시
2451,00000A33166,기타 화학제품 제조업,2019-10-28,경기도


In [27]:
ki2=ki.copy() #사본

In [28]:
ki2.상품번호=ki2.상품번호.apply(lambda x:x[:-1]) #우선주 보통주 구분 없이 통합하기 위해 맨 뒤번호 제거
ki2.상품번호

0         00000A09197
1         00000A04506
2         00000A25384
3         00000A00919
4         00000A00174
             ...     
845935    00000A04320
845936    00000A06876
845937    00000A03829
845938    00000A00066
845939    00000A00569
Name: 상품번호, Length: 845940, dtype: object

In [38]:
ki3=pd.merge(ki2,st3,how='left',on='상품번호')

In [39]:
ki3

,고객구분코드,주문일자,고객성별구분코드,동일나이군구분코드,주소(시도),주문시간대,상품번호,상품명,거래소구분코드,매도매수구분코드,...,일중고가,일중저가,당일종가,거래수량,거래대금,상한가여부,하한가여부,업종,상장일,지역
0,A0001,20200128,1,30,경기,1,00000A09197,나노캠텍,3,2,...,4125,3660,4095,17266597,67323312855,N,N,플라스틱제품 제조업,2007-01-30,경기도
1,A0001,20200130,1,30,경기,3,00000A04506,오공,3,2,...,8380,7360,7920,15686931,125404000000,N,N,기타 화학제품 제조업,2000-08-10,인천광역시
2,A0001,20200313,1,30,경기,1,00000A25384,수젠텍,3,2,...,12250,10150,10750,3731106,40782765600,N,N,의료용품 및 기타 의약 관련제품 제조업,2019-05-28,대전광역시
3,A00010,20200220,1,20,경남,3,00000A00919,대양금속보통주,2,2,...,3675,3475,3505,635410,2250893355,N,N,1차 철강 제조업,1994-04-29,충청남도
4,A000100,20200211,2,40,서울,1,00000A00174,SK네트웍스보통주,2,1,...,4885,4800,4850,1263089,6112026870,N,N,기타 전문 도매업,1977-06-30,경기도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845935,A0009999,20200331,1,20,충북,2,00000A04320,파루,3,2,...,2135,1990,2030,2764564,5682482755,N,N,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업",2000-07-19,전라남도
845936,A0009999,20200325,1,20,충북,2,00000A06876,셀트리온제약,3,1,...,70600,58700,60300,8892944,560929000000,N,N,의약품 제조업,2006-02-03,충청북도
845937,A0009999,20200330,1,20,충북,1,00000A03829,마크로젠,3,2,...,40800,37500,40400,1912985,75631708300,N,N,"그외 기타 전문, 과학 및 기술 서비스업",2000-02-22,서울특별시
845938,A0009999,20200323,1,20,충북,1,00000A00066,에스케이하이닉스보통주,2,1,...,71600,68000,69400,8037426,560505000000,N,N,반도체 제조업,1996-12-26,경기도


In [40]:
ki3.isnull().sum()

고객구분코드          0
주문일자            0
고객성별구분코드        0
동일나이군구분코드       0
주소(시도)          0
주문시간대           0
상품번호            0
상품명             0
거래소구분코드         0
매도매수구분코드        0
주문구분코드          0
실주문단가           0
주문수량            0
총체결수량           0
총체결금액           0
전일종가            0
당일시가            0
일중고가            0
일중저가            0
당일종가            0
거래수량            0
거래대금            0
상한가여부           0
하한가여부           0
업종           2128
상장일          2128
지역           2128
dtype: int64

In [56]:
ki3[ki3['업종'].isnull()].상품명.unique()


array(['제낙스', '썬테크놀로지스', '아이엠텍', '에이팸', '두산건설보통주', '미래SCI', '지에스홈쇼핑',
       '퓨전', '코리아오토글라스보통주', '에스케이씨 솔믹스', '고려개발보통주', '케이맥', '동성화학보통주',
       '한국제지보통주', '럭슬'], dtype=object)

In [48]:
ki3.groupby('거래소구분코드')['주문일자'].count()

거래소구분코드
2    391077
3    454863
Name: 주문일자, dtype: int64

In [7]:
ki3=ki2.loc[ki.동일나이군구분코드.isin([20,30]),] # 20대 30대

In [8]:
ki3

,고객구분코드,주문일자,고객성별구분코드,동일나이군구분코드,주소(시도),주문시간대,상품번호,상품명,거래소구분코드,매도매수구분코드,...,총체결금액,전일종가,당일시가,일중고가,일중저가,당일종가,거래수량,거래대금,상한가여부,하한가여부
0,A0001,20200128,1,30,경기,1,00000A091970,나노캠텍,3,2,...,75900,3215,3710,4125,3660,4095,17266597,67323312855,N,N
1,A0001,20200130,1,30,경기,3,00000A045060,오공,3,2,...,154200,7690,8150,8380,7360,7920,15686931,125404000000,N,N
2,A0001,20200313,1,30,경기,1,00000A253840,수젠텍,3,2,...,118000,13600,12200,12250,10150,10750,3731106,40782765600,N,N
3,A00010,20200220,1,20,경남,3,00000A009190,대양금속보통주,2,2,...,49070,3615,3615,3675,3475,3505,635410,2250893355,N,N
10,A0001000,20200318,1,30,광주,3,00000A012690,모나리자보통주,2,2,...,39720,4570,4600,5280,4550,4700,11346291,55836926545,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845935,A0009999,20200331,1,20,충북,2,00000A043200,파루,3,2,...,303000,1995,2035,2135,1990,2030,2764564,5682482755,N,N
845936,A0009999,20200325,1,20,충북,2,00000A068760,셀트리온제약,3,1,...,507200,57300,61700,70600,58700,60300,8892944,560929000000,N,N
845937,A0009999,20200330,1,20,충북,1,00000A038290,마크로젠,3,2,...,970000,38900,40050,40800,37500,40400,1912985,75631708300,N,N
845938,A0009999,20200323,1,20,충북,1,00000A000660,에스케이하이닉스보통주,2,1,...,553600,74800,69700,71600,68000,69400,8037426,560505000000,N,N


In [9]:
ki3.groupby('상품명')['상품번호'].count()

상품명
(주)마이크로컨텍솔루션          18
(주)스카이이앤엠             69
(주)포스코엠텍              39
AJ네트웍스보통주             76
AK홀딩스보통주              91
                    ... 
흥국화재해상보험1우선주         162
흥국화재해상보험2우선주(신형)      43
흥국화재해상보험보통주           82
흥아해운보통주             1168
힘스                    66
Name: 상품번호, Length: 1966, dtype: int64